<a href="https://colab.research.google.com/github/5koe/BOSTONDATA/blob/main/notebooks/regression/forward_selection_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
!git clone https://github.com/pharringtonp19/business-analytics.git

fatal: destination path 'business-analytics' already exists and is not an empty directory.


### **Import Packages**

In [107]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import jax.numpy as jnp
import jax
import seaborn as sb
from functools import partial

### **Specify Dataset**

In [108]:
dataset = 'brookline'

### **Read In Data Set**

In [109]:
if dataset == 'brookline':
  df = pd.read_csv('/content/business-analytics/datasets/brookline.csv')
else:
  df = pd.read_csv('/content/business-analytics/datasets/CASchools.csv')
  df['Students_per_Teacher'] = df['students'] / df['teachers']
df.head()

,price,stNumber,stName,size,beacon,baseFloor,buildingStyle,elevators,rooms,bedrooms,fullBathrooms,halfBathrooms,garage
0,174000,150,PLEASANT ST,1060,0,4,MID-RISE,1,4,2,1,1,1.0
1,337000,7,LEVERETT ST,831,0,1,DECKER,0,4,2,1,0,0.0
2,850000,24,EUSTON ST,2246,0,1,ROW-END,0,10,6,3,0,0.0
3,516000,417,WASHINGTON ST,1574,0,2,LOW-RISE,0,6,3,2,0,0.0
4,145000,150,PLEASANT ST,669,0,4,MID-RISE,1,3,1,1,0,1.0


### **Create Function**

In [110]:
def get_rsqr(reg_formula):
  linear_model = smf.ols(reg_formula, data=df)
  results = linear_model.fit()
  return results.rsquared_adj, results.rsquared

get_rsqr('price ~ size')

(0.7488035137772363, 0.7490352448530876)

### **Store Results**

In [111]:
ordered_variables = []
optimal_adj_rsqur, optimal_rsqr = [], []

### **Specify the dependent variable**

In [112]:
dep_var = 'price'

### **Create list of possible Right Hand Side Variables**

In [113]:
eligible_vars = df.columns.drop(['stNumber', 'stName', dep_var]).tolist()
categorical_vars = ['buildingStyle']
eligible_vars

['size',
 'beacon',
 'baseFloor',
 'buildingStyle',
 'elevators',
 'rooms',
 'bedrooms',
 'fullBathrooms',
 'halfBathrooms',
 'garage']

### **Create Dictionary of Variable Name and Regression String Format**

In [114]:
eligible_vars = {key: key if key not in categorical_vars else f'C({key})' for key in eligible_vars}
eligible_vars

{'size': 'size',
 'beacon': 'beacon',
 'baseFloor': 'baseFloor',
 'buildingStyle': 'C(buildingStyle)',
 'elevators': 'elevators',
 'rooms': 'rooms',
 'bedrooms': 'bedrooms',
 'fullBathrooms': 'fullBathrooms',
 'halfBathrooms': 'halfBathrooms',
 'garage': 'garage'}

### **Create Regression String**

In [115]:
reg_formula = dep_var + ' ~ '
reg_formula

'price ~ '

### **For Loop**

In [116]:
for i in range(len(eligible_vars)):

  if i == 0:
    reg_formula = list(map(lambda x: reg_formula + x, eligible_vars.values()))
    # else
    reg_formula = list(map(lambda x: reg_formula + ' + ' + x, eligible_vars.values()))

results = np.array(list(map(get_rsqr, reg_formula)))
adj_rsqrs = results[:, 0]

argmax_id = np.argmax(adj_rsqrs)
new_rhs = list(eligible_vars.keys)

TypeError: can only concatenate list (not "str") to list

In [ ]:
for i in range(len(eligible_vars)):

  if i == 0:
    reg_formulas = list(map(lambda x: reg_formulas + x, eligible_vars.values()))
  else:
    reg_formulas = list(map(lambda x: reg_formulas + " + " + x, eligible_vars.values()))

results = np.array(list(map(get_rsqr, reg_formulas)))

  # Create a List of regression strings (one for each possible new independent variable)

  # Compute the Rsquared and Adjusted-Rsquared for each regression model

  # Extract the adjusted_rsquared_array (first column of results)

  # Find the index with the largest adjusted_rsquared

  # Use the index to select the corresponding indpendent variable with the largest adjusted_rsquared

  # Append the Variable to the Ordered Variable List

  # Update the Regression Formula

  # Update the list of potential independent variables

  # Use the index to retrieve the adjusted_rsquared and r_squared associated with that index

  # Append the values

### **Plot**

In [ ]:
# Create the figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Define width for bars and positions for each pair of bars
bar_width = 0.35
x = np.arange(len(ordered_variables))

# Plot adjusted R-squared and R-squared bars side by side
bars_adj = ax.bar(x - bar_width / 2, optimal_adj_rsqr, width=bar_width, label="Adjusted", alpha=0.7)
bars_rs = ax.bar(x + bar_width / 2, optimal_rsqr, width=bar_width, label="Default", alpha=0.5)

# Add labels, title, and angle the x-axis labels
ax.set_title(r"$R^2$", loc='left')
ax.set_xticks(x)
ax.set_xticklabels(ordered_variables, rotation=45, ha='right')
ax.legend(loc='center left', bbox_to_anchor=(.95, 0.9))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Display values above each bar
for bar in bars_adj:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')

for bar in bars_rs:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')

plt.show()